In [1]:
# for PyCharm
import os

current_directory = os.getcwd()
project_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
os.chdir(project_directory)

In [2]:
import os

import pandas as pd
from pathlib import Path
import numpy as np
from src.data.merger import SuperkicksFormatter, SneakerbaasFormatter, FootshopFormatter, HighsnobietyFormatter, \
    KickscrewFormatter



In [3]:
metadata_path = "data/raw/metadata"
discovered_datasets = os.listdir(metadata_path)
discovered_datasets

['superkicks.csv',
 'highsnobiety.csv',
 'footshop.csv',
 'kickscrew.csv',
 'sneakerbaas.csv']

In [5]:
try:
    datasets = {
        source: pd.read_csv(str(Path(metadata_path, source))) for source in discovered_datasets
    }
    assert len(datasets) > 0
except FileNotFoundError as err:
    print(f"Some dataset could not be resolved:")
    raise

In [7]:
datasets["sneakerbaas.csv"]

,brand,description,pricecurrency,price,title,collection_name,collection_url,url,images_path
0,new balance,"- New Balance 550 ""Vintage Teal""- Colour: Whit...",EUR,79.99,550 vintage teal,category-kids,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-kids/new ...
1,new balance,"- New Balance 550 ""WHITE""- Colour: White/Yello...",EUR,79.99,550 white,category-kids,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-kids/new ...
2,nike,Stylecode: FN7788-100,EUR,49.99,blazer mid next nature white,category-kids,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-kids/nike...
3,nike,"- NIKE AIR MAX 90 GS ""BLACK/HABANERO-SMOKE GRE...",EUR,89.99,air max 90 gs habanero,category-kids,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-kids/nike...
4,vans,"- UY SK8-Mid Reissue V ""CTHR CHB""- Colour: Blu...",EUR,39.99,sk8-mid checkerboard,category-kids,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-kids/vans...
...,...,...,...,...,...,...,...,...,...
1000,adidas originals,A Clean Classic. These men's Supercourt Shoes ...,EUR,69.99,supercourt white,category-men,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-men/adida...
1001,clarks,"- Clarks Desert Boot ""Purple""- Colour: Purple-...",EUR,79.99,desert boot purple,category-men,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-men/clark...
1002,karhu,"- Karhu Legacy OG ""Purple""- Colour: White/Purp...",EUR,69.99,legacy og purple,category-men,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-men/karhu...
1003,karhu,"- Karhu Synchron Classic ""Gray Violet""- Stylec...",EUR,69.99,synchron classic violet,category-men,https://www.sneakerbaas.com/collections/sneake...,https://www.sneakerbaas.com/collections/sneake...,data/raw/images/sneakerbaas/category-men/karhu...


In [1]:
# todo
# Я перепарсил и теперь в title есть цвет в кавычках типо вот этого: Air Jordan 6 Retro GS "Dutch Blue"
# его бы вынести в отдельную колонку color, это надо в merger.py переписать для sneakerbaas и superkicks ну и ваших датасетов

# После этого надо сделать колонку, в которой будет чистое название (без цвета и лишних символов)
# мб ебануть AbstractParser.remove_extra_symbols() на title без цветов.
# По этой чистой колонке можно смерджить.
raise NotImplementedError


NameError: name 'SuperkicksFormatter' is not defined

In [ ]:
processed_datasets = {
    "superkicks": SuperkicksFormatter(datasets["superkicks.csv"]).format(),
    "sneakerbaas": SneakerbaasFormatter(datasets["sneakerbaas.csv"]).format(),
    "footshop": FootshopFormatter(datasets["footshop.csv"]).format(),
    "highsnobiety": HighsnobietyFormatter(datasets["highsnobiety.csv"]).format(),
    "kickscrew": KickscrewFormatter(datasets["kickscrew.csv"]).format(),
}
# check_extra_symbols(processed_datasets)

In [6]:
for key, value in processed_datasets.items():
    print(value.shape)

(1090, 10)
(1005, 10)
(4864, 10)
(598, 8)
(4900, 8)


In [7]:
# same columns
same_columns = set.intersection(*[set(processed_datasets[key].columns) for key in processed_datasets])
same_columns
# no colors and collection-name

{'brand',
 'images_path',
 'price',
 'pricecurrency',
 'title',
 'title_old',
 'url',
 'website'}

In [8]:
merged_datasets = pd.concat(list(processed_datasets.values()), ignore_index=True)
merged_datasets

,brand,title,price,collection_name,url,images_path,website,title_old,pricecurrency,color
0,converse,wmns run star legacy cx,8499.0,men-sneakers,https://www.superkicks.in/products/wmns-run-st...,data/raw/images/superkicks/men-sneakers/conver...,superkicks,wmns run star legacy cx periwinkle,INR,[]
1,converse,chuck taylor all star,4299.0,men-sneakers,https://www.superkicks.in/products/chuck-taylo...,data/raw/images/superkicks/men-sneakers/conver...,superkicks,chuck taylor all star charcoal,INR,[]
2,nike,kd16 nrg ep,14995.0,men-sneakers,https://www.superkicks.in/products/kd16-nrg-ep...,data/raw/images/superkicks/men-sneakers/nike/k...,superkicks,kd16 nrg ep pink foam|pink|white,INR,[]
3,jordan,air jordan 1 retro high og royal bluewhiteroyal,16995.0,men-sneakers,https://www.superkicks.in/products/air-jordan-...,data/raw/images/superkicks/men-sneakers/jordan...,superkicks,air jordan 1 retro high og black|royal blue-wh...,INR,[]
4,nike,blazer mid 77 premium summit blacklight,7756.0,men-sneakers,https://www.superkicks.in/products/blazer-mid-...,data/raw/images/superkicks/men-sneakers/nike/b...,superkicks,blazer mid 77 premium summit white|black-light...,INR,[]
...,...,...,...,...,...,...,...,...,...,...
12452,adidas,adidas ultraboost 10 shoes valentines day,123.0,NaN,https://www.kickscrew.com/products/adidas-ultr...,[data/raw/images/kickscrew/adidas/adidas-ultra...,kickscrew,Adidas Ultraboost 1.0 Shoes 'Valentines Day',USD,NaN
12453,adidas,adidas originals ozweego shoes magic clay strata,102.0,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,Adidas Originals Ozweego Shoes 'Magic Beige Cl...,USD,NaN
12454,adidas,adidas originals adilette slides off bright,56.0,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,Adidas Originals Adilette Slides 'Off White Br...,USD,NaN
12455,adidas,adidas originals forum low shoes,84.0,NaN,https://www.kickscrew.com/products/adidas-orig...,[data/raw/images/kickscrew/adidas/adidas-origi...,kickscrew,Adidas Originals Forum Low Shoes 'White Grey Red',USD,NaN


In [9]:
# group by по названиям, если все названия будут одинаковые, то все дублирующие кроссы сложатся в одно.
test = merged_datasets.groupby("title").agg({
    "brand": list,
    "collection_name": list,
    "color": list,
    "images_path": list,
    "price": list,
    "pricecurrency": list,
    "url": list,
    "website": list
}).reset_index().sort_values(by="title")
test

,title,brand,collection_name,color,images_path,price,pricecurrency,url,website
0,01 low,"[autry, autry]","[category-women, category-men]","[[white, beige], [yellow, white]]",[data/raw/images/sneakerbaas/category-women/au...,"[129.99, 149.99]","[EUR, EUR]",[https://www.sneakerbaas.com/collections/sneak...,"[sneakerbaas, sneakerbaas]"
1,01 low m,[autry],[category-men],"[[black, white, creme]]",[data/raw/images/sneakerbaas/category-men/autr...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
2,01 low m bi,[autry],[category-men],"[[white, purple]]",[data/raw/images/sneakerbaas/category-men/autr...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
3,01 low m bi eden,[autry],[category-men],"[[white, green]]",[data/raw/images/sneakerbaas/category-men/autr...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
4,01 low w bi,[autry],[category-women],"[[black, white]]",[data/raw/images/sneakerbaas/category-women/au...,[149.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
...,...,...,...,...,...,...,...,...,...
8233,zoom vomero 5 sp anthracite anthraciteblackwolf,[nike],[men-sneakers],[[]],[data/raw/images/superkicks/men-sneakers/nike/...,[14995.0],[INR],[https://www.superkicks.in/products/zoom-vomer...,[superkicks]
8234,zoom vomero 5 sp vast vast greyblacksail,[nike],[men-sneakers],[[]],[data/raw/images/superkicks/men-sneakers/nike/...,[14995.0],[INR],[https://www.superkicks.in/products/zoom-vomer...,[superkicks]
8235,zx 22 boost,[adidas originals],[category-men],[[beige]],[data/raw/images/sneakerbaas/category-men/adid...,[59.99],[EUR],[https://www.sneakerbaas.com/collections/sneak...,[sneakerbaas]
8236,zx 2k boost 20,[adidas],[women-sneakers],[[]],[data/raw/images/superkicks/women-sneakers/adi...,[8399.0],[INR],[https://www.superkicks.in/products/zx-2k-boos...,[superkicks]


In [10]:
# sizes
print(merged_datasets.shape)
print(test.shape)

(12457, 10)
(8238, 9)


In [11]:
titles = test["title"].tolist()

In [12]:
!pip install thefuzz -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
from thefuzz import fuzz

def similarity(string1, string2):
    return fuzz.token_sort_ratio(string1, string2)

title = "vans - ua og authentic lx yellow"

similarities = []
for text in titles:
    similarities.append((text, similarity(title, text)))

similarities.sort(key=lambda x: x[1])
similarities[-10:]

[('vans ua og authentic lx vault checkerboard', 69),
 ('vans vault authentic one piece vlt lx', 69),
 ('vans vault og authentic frayed lx', 70),
 ('vans vault og authentic lx canvas', 70),
 ('authentic los vans', 71),
 ('vans og authentic lx stressed', 71),
 ('vans ua authentic vr3 pw lx', 77),
 ('vans ua og authentic lx suede', 78),
 ('vans vault og authentic lx', 79),
 ('vans og authentic lx', 80)]

In [18]:
# эта тема лучше всего находит похожие

In [14]:
test.to_csv("temp.csv")